In [1]:
from ArgoData.data_centerline import Argo2Dataset
from models.ganet import get_model
from torch.utils.data import DataLoader
from av2.datasets.motion_forecasting.eval.submission import ChallengeSubmission, ScenarioPredictions
import torch.nn as nn
import torch

In [2]:
config, collate_fn, net, loss, post_process, opt = get_model()
checkpoint = torch.load("./models/results/ganet1/36.000.ckpt")
net.load_state_dict(checkpoint['state_dict'], strict = False)   
m = nn.Softmax(dim=0)

test_data = Argo2Dataset(root = '../', split  = 'val')

test_loader = DataLoader(
    test_data,
    batch_size = 1,
    num_workers = config['val_workers'],
    shuffle = False,
    collate_fn = collate_fn,
    pin_memory=True,
)


In [3]:
for i, test_data in enumerate(test_loader):
    if i > 1:
        break

In [4]:
goal, out = net(test_data)
gt_preds = test_data['gt_preds']
has_preds = test_data['has_preds']

cls, reg = out["cls"], out["reg"]
cls = torch.cat([x for x in cls], 0)
reg = torch.cat([x for x in reg], 0)
gt_preds = torch.cat([x for x in gt_preds], 0)
gt_preds = gt_preds.to(reg.device)
has_preds = torch.cat([x for x in has_preds], 0)
num_mods, num_preds = config["num_mods"], config["num_preds"]
last = has_preds.float() + 0.1 * torch.arange(num_preds).float().to(has_preds.device)/float(num_preds)


max_last, last_idcs = last.max(1) 
mask = max_last > 1.0

# valid final cls scores
cls = cls[mask]
# valid final prediction positions
reg = reg[mask]
gt_preds = gt_preds[mask]
has_preds = has_preds[mask]
last_idcs = last_idcs[mask]

In [5]:
row_idcs = torch.arange(len(last_idcs)).long().to(last_idcs.device)
dist = []
for j in range(num_mods):
    dist.append(
        torch.sqrt(
            (
                (reg[row_idcs, j, last_idcs] - gt_preds[row_idcs, last_idcs])
                ** 2
            ).sum(1)
        )
    )

In [23]:
reg[row_idcs, j, last_idcs].shape

torch.Size([18, 2])